<a href="https://colab.research.google.com/github/Sourav-Manik/AI_Deep_Learning/blob/main/Hyperparameter_Optimization_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1.Import Library##

In [ ]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tf.__version__

'2.9.2'

##2.Import Data##

In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


##3.Data Understanding##

In [ ]:
df.shape

(768, 9)

In [ ]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [ ]:
df.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

##4.Data Preprocessing || 5. Model Building##

In [ ]:
X = df.drop('Outcome',axis = 1)
y= df[['Outcome']]

In [ ]:
std_scaler = StandardScaler()
std_scaler = std_scaler.fit_transform(X)
X_scaled = pd.DataFrame(data = std_scaler,columns = X.columns)
X_scaled

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=0.20,stratify = y, random_state=12, shuffle = True)

In [ ]:
X_train.shape, y_train.shape

((614, 8), (614, 1))

In [ ]:
X_test.shape, y_test.shape

((154, 8), (154, 1))

#6.Hyperparameter Tunning

##6.1. Learning Rate and Dropout Rate##

In [ ]:
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(units=  8, input_dim = 8, kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=  12,input_dim = 8, kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=  1,activation = 'sigmoid'))

    model.compile(optimizer = tf.optimizers.Adam(learning_rate=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
    return model

Create the Model

In [ ]:
model = KerasClassifier(build_fn = create_model,verbose = 0, batch_size=20,epochs = 30)

Define the grid search parameters

In [ ]:
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

Make a dictionary of the grid search parameters

In [ ]:
param_grids = dict(learning_rate = learning_rate, dropout_rate = dropout_rate)

Build and fit the GridSearchCV

In [ ]:
grid = GridSearchCV(estimator = model,param_grid = param_grids, cv = KFold(),verbose = 10)
grid_result = grid.fit(X_scaled,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.760 total time=   8.6s
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.727 total time=   3.2s
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.753 total time=   3.4s
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.856 total time=   2.7s
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.752 total time=   2.8s
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 1/5; 

In [ ]:
grid_result.best_params_

{'dropout_rate': 0.1, 'learning_rate': 0.001}

Summarize the results

In [ ]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))

Best : 0.7709192752838134, using {'dropout_rate': 0.1, 'learning_rate': 0.001}


In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
print(means)

[0.76962057 0.75661659 0.72661915 0.77091928 0.75141331 0.74357016
 0.76442579 0.76572448 0.72658519]


In [ ]:
for mean, stdev, param in zip(means,stds,params):
  print('{},{} with: {}'.format(mean,stdev,param))

0.7696205735206604,0.044676978808197244 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
0.7566165924072266,0.04545515386026056 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
0.7266191482543946,0.034768657403786936 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
0.7709192752838134,0.038494168969296735 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.7514133095741272,0.042863473215584307 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.7435701608657836,0.051064611581257253 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.764425790309906,0.04377881074447207 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.7657244801521301,0.04130242668152803 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.7265851855278015,0.02638550973178768 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


##6.2. Activation Function and Kernel Initializer

Define the model

In [ ]:
def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(units=  8, input_dim = 8, kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(units=  12, input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(units=  1,activation = 'sigmoid'))

    model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


Create the model

In [ ]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

Define the grid search parameters

In [ ]:
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

Make a dictionary of the grid search parameters

In [ ]:
param_grids = dict(activation_function = activation_function, init = init)

Build and fit the GridSearchCV

In [ ]:
grid = GridSearchCV(estimator = model,param_grid = param_grids, cv = KFold(),verbose = 10)
grid_result = grid.fit(X_scaled,y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 1/5; 1/12] END activation_function=softmax, init=uniform;, score=0.649 total time=   2.7s
[CV 2/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 2/5; 1/12] END activation_function=softmax, init=uniform;, score=0.584 total time=   2.7s
[CV 3/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 3/5; 1/12] END activation_function=softmax, init=uniform;, score=0.630 total time=   3.5s
[CV 4/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 4/5; 1/12] END activation_function=softmax, init=uniform;, score=0.745 total time=   2.7s
[CV 5/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 5/5; 1/12] END activation_function=softmax, init=uniform;, score=0.647 total time=   3.2s
[CV 1/5; 2/12] START activation_function=softmax, init=normal...

In [ ]:
grid_result.best_params_

{'activation_function': 'relu', 'init': 'normal'}

Summarize the results

In [ ]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))

Best : 0.7735166788101197, using {'activation_function': 'relu', 'init': 'normal'}


In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
print(means)

[0.65115864 0.65115864 0.65115864 0.76702317 0.77351668 0.65115864
 0.76698922 0.76829641 0.65115864 0.76698922 0.7696036  0.65115864]


In [ ]:
for mean, stdev, param in zip(means,stds,params):
  print('{},{} with: {}'.format(mean,stdev,param))

0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'init': 'uniform'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'init': 'zero'}
0.7670231699943543,0.043389198132676055 with: {'activation_function': 'relu', 'init': 'uniform'}
0.7735166788101197,0.03604050561258882 with: {'activation_function': 'relu', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'relu', 'init': 'zero'}
0.7669892191886902,0.036149489629613486 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.7682964086532593,0.03568269945545852 with: {'activation_function': 'tanh', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'tanh', 'init': 'zero'}
0.7669892191886902,0.03587539107361029 with: {'activation_function': 'linear', 'init': 'uniform'}
0.7696035981178284,0.033407053079046296

##6.3. Number of Neurons in Activation Layer

Define the model

In [ ]:
def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(units=  neuron1, input_dim = 8, kernel_initializer = 'uniform' ,activation = 'linear'))
    model.add(Dropout(0.2))
    model.add(Dense(units=  neuron2, input_dim=neuron1, kernel_initializer = 'uniform' ,activation = 'linear'))
    model.add(Dropout(0.2))
    model.add(Dense(units=  1,activation = 'sigmoid'))

    model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


Create the model

In [ ]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

Define the grid search parameters

In [ ]:
neuron1= [4,8,16]
neuron2=[2,4,8]

Make a dictionary of the grid search parameters

In [ ]:
param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

Build and fit the GridSearchCV

In [ ]:
grid = GridSearchCV(estimator = model,param_grid = param_grids, cv = KFold(),verbose = 10)
grid_result = grid.fit(X_scaled,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.779 total time=   2.7s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.721 total time=   3.0s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.766 total time=   2.6s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.837 total time=   2.7s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.745 total time=   3.1s
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END .........neuron1=4, neuron2=4;,

Summarize the results

In [ ]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))

Best : 0.7734997034072876, using {'neuron1': 8, 'neuron2': 2}


In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means,stds,params):
  print('{},{} with: {}'.format(mean,stdev,param))

0.7695866227149963,0.038927826086520435 with: {'neuron1': 4, 'neuron2': 2}
0.7669807314872742,0.036865550404712115 with: {'neuron1': 4, 'neuron2': 4}
0.7695951104164124,0.03372031359429817 with: {'neuron1': 4, 'neuron2': 8}
0.7734997034072876,0.03588668553922825 with: {'neuron1': 8, 'neuron2': 2}
0.7656905174255371,0.037375792829326994 with: {'neuron1': 8, 'neuron2': 4}
0.7696120858192443,0.036947040093127693 with: {'neuron1': 8, 'neuron2': 8}
0.7669977068901062,0.039861653084249986 with: {'neuron1': 16, 'neuron2': 2}
0.7708938121795654,0.0313219899987568 with: {'neuron1': 16, 'neuron2': 4}
0.764383327960968,0.033427846560061106 with: {'neuron1': 16, 'neuron2': 8}


# 7.Training Model with Optimal values of Hyperparameter

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

Define the model

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(units=  16, input_dim = 8, kernel_initializer = 'uniform' ,activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(units=  4, input_dim=16, kernel_initializer = 'uniform' ,activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(units=  1,activation = 'sigmoid'))

    model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

Create Model

In [ ]:
model = KerasClassifier(build_fn = create_model, verbose = 0,batch_size=40, epochs = 10)

Fitting the model

In [ ]:
model.fit(X_scaled,y)

Predicting using trained model

In [ ]:
y_predict = model.predict(X_scaled)

24/24 [==============================] - 0s 1ms/step


Printing the metrics

In [ ]:
print(accuracy_score(y,y_predict))

0.7721354166666666


The hyperparameter optimization was carried out by taking 2 hyperparameters at once. We may have missed the best values. The performance can be further improved by finding the optimum values of hyperparameters all at once. Please check 'Hyperparameter_Optimization_3 (All at once).ipynb' file.

In [ ]:
#End